<a href="https://colab.research.google.com/github/ellieytc/CaseStudy.DeepLearning/blob/master/food_train_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Thu Aug 20 10:48:20 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.57       Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   72C    P8    35W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import os
import zipfile
from google.colab import drive
drive.mount('/content/drive')

zip_ref = zipfile.ZipFile('/content/drive/My Drive/Tibame/dataset/food_image.zip', 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

os.chdir('/tmp')
os.listdir('/tmp')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


['drivefs_ipc.0_shell',
 'tmpm5wdu9lb',
 'food_train',
 'food_test',
 'drivefs_ipc.0']

In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import random
import os

from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from cv2 import cv2

from keras.models import Sequential
from keras import layers
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation,GlobalMaxPooling2D
from keras import applications
from keras.applications.resnet50 import ResNet50
from keras import backend as K
from keras.preprocessing.image import ImageDataGenerator, load_img
from keras import optimizers
from keras.models import Model
import keras

# prepare training data and labels 
filenames = os.listdir('/tmp/food_train/')
categories = []
for filename in filenames:
    category = filename.split('_')[0]
    if category == 'apple':
      categories.append(0)
    elif category == 'cabbage':
      categories.append(1)
    elif category == 'carrot':
      categories.append(2)
    elif category == 'chicken':
      categories.append(3)
    elif category == 'cucumber':
      categories.append(4)
    elif category == 'egg':
      categories.append(5)
    elif category == 'mushroom':
      categories.append(6)
    elif category == 'potato':
      categories.append(7)
    elif category == 'radish':
      categories.append(8)       
    else:
      categories.append(9)

df = pd.DataFrame({'filename': filenames,'category': categories}, dtype=str)

# split train/test data
train_df, validate_df = train_test_split(df, test_size=0.1)
train_df = train_df.reset_index()
validate_df = validate_df.reset_index()
total_train = train_df.shape[0]
total_validate = validate_df.shape[0]

# parameters
image_size = (200, 200)
num_classes = 10
freeze_layers = 2
epochs = 5
batch_size = 16

# data augmentation
train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1)

train_generator = train_datagen.flow_from_dataframe(
    train_df, 
    '/tmp/food_train/', 
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=image_size,
    batch_size=batch_size)

validation_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = validation_datagen.flow_from_dataframe(
    validate_df, 
    '/tmp/food_train/', 
    x_col='filename',
    y_col='category',
    class_mode='categorical',
    target_size=image_size,
    batch_size=batch_size)

# use ResNet50 model as baseline and remove the fully connected layers
pre_trained_model = ResNet50(include_top=False, 
                             weights='imagenet',
                             input_tensor=None,
                             input_shape=(image_size[0],image_size[1],3))

x = pre_trained_model.output
x = Flatten()(x)
x = Dropout(0.5)(x)

# add Dense layer and softmax
output_layer = Dense(num_classes, activation='softmax', name='softmax')(x)

# set freeze layers and trained layers
model = Model(inputs=pre_trained_model.input, outputs=output_layer)
for layer in pre_trained_model.layers[:freeze_layers]:
    layer.trainable = False
for layer in pre_trained_model.layers[freeze_layers:]:
    layer.trainable = True

# compile model
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.SGD(lr=1e-4, momentum=0.9),
              metrics=['accuracy'])
print(model.summary())

# fine-tune the model
history = model.fit_generator(train_generator,
                              epochs=epochs,
                              validation_data=validation_generator,
                              validation_steps=total_validate//batch_size,
                              steps_per_epoch=total_train//batch_size)

Found 4995 validated image filenames belonging to 10 classes.
Found 555 validated image filenames belonging to 10 classes.
94773248/94765736 [==============================] - 2s 0us/step
Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 200, 200, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 206, 206, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 100, 100, 64) 9472        conv1_pad[0][0]                  
_________________________________________________________________________________________________

In [5]:
# save model
model.save('model-food-final.h5')
import pickle
with open('food_train_model.pkl' , 'wb') as file:
  pickle.dump('model-food-final.h5', file)